In [ ]:
# Functions for Creating Hamiltonian and Running Simulation

using ITensors, ITensorMPS
using LinearAlgebra
using DataFrames, CSV
using Plots

function make_H0(L, sites, bc)
    osI = OpSum()
    for j in 1:L
        osI -= "Z", j
    end   
    for j in 1:L-1
        osI -= "X", j, "X", j+1
    end
    
    if bc == "PBC"
        osI -= "X", L, "X", 1
    elseif bc == "APBC"
        osI += "X", L, "X", 1
    end

    return MPO(osI, sites)
end

function make_H3(L, sites, bc)
    os3 = OpSum()
    for j in 1:L-2
        os3 += "Z", j, "X", j+1, "X", j+2
        os3 += "X", j, "X", j+1, "Z", j+2
    end

    if bc == "PBC"
        os3 += "Z", L-1, "X", L, "X", 1
        os3 += "X", L-1, "X", L, "Z", 1
        os3 += "Z", L, "X", 1, "X", 2
        os3 += "X", L, "X", 1, "Z", 2
    elseif bc == "APBC"
        os3 -= "Z", L-1, "X", L, "X", 1
        os3 += "X", L-1, "X", L, "Z", 1
        os3 += "Z", L, "X", 1, "X", 2
        os3 -= "X", L, "X", 1, "Z", 2
    end

    return MPO(os3, sites)
end

function local_H(t0, g, s1, s2, s3; signs=[1.0,1.0,1.0,1.0])

    H = 2*t0*(-signs[1]*op("Z", s1)*op("Id", s2)*op("Id", s3))

    if signs[2] != 0
        H += 2*t0*(-signs[2]*op("X", s1)*op("X", s2)*op("Id", s3))
    end

    if signs[3] != 0
        H += g*signs[3]*op("Z", s1)*op("X", s2)*op("X", s3)
    end
    if signs[4] != 0
        H += g*signs[4]*op("X", s1)*op("X", s2)*op("Z", s3)
    end

    return H
end

function build_trotter_gates(sites, tau, t0, g, bc)

    L = length(sites)
    gates = ITensor[]
    
    for j in 1:L-2
        H = local_H(t0, g, sites[j], sites[j+1], sites[j+2])
        push!(gates, exp(-im*tau/2 * H))
    end
    
    append!(gates, reverse(gates))
    return gates
end
nothing

In [ ]:
# Running Time-Indepent DMRG

L = 4
bc = "OBC" # Boundary Conditions: Open = "OBC", Periodic = "PBC", Anti-Periodic = "APBC"
t0 = 1
g = 1
parity = 1
nsweeps = 20
cutoff = 1e-8
maxdim = 100
dmrg_tol = 1e-8

sites = siteinds("S=1/2", L, conserve_szparity=true)

H0 = make_H0(L, sites, bc)
H3 = make_H3(L, sites, bc)
H = 2*t0*H0 + g*H3

state = ["Up" for _ in 1:L]
if parity == -1
    state[1] = "Dwn"
end

psi = random_mps(sites, state, linkdims=4)

observer = DMRGObserver(energy_tol=dmrg_tol)
maxdim = [10, 50, 100, 500, 1000, 5000, 10000, 50000]

E0, psi0 = dmrg(H, psi; nsweeps, maxdim, cutoff, observer, outputlevel=1, noise=[0.0], eigsolve_krylovdim=10)

println("Ground state energy: ", E0)
println("Overlap: ", inner(psi0, psi0))
println("Final Maximum Bond Dimension: ", maxlinkdim(psi0))

After sweep 1 energy=-8.320148853431087  maxlinkdim=4 maxerr=9.24E-16 time=0.007
After sweep 2 energy=-8.320148853431087  maxlinkdim=4 maxerr=1.15E-15 time=0.004
After sweep 3 energy=-8.320148853431089  maxlinkdim=4 maxerr=8.13E-16 time=0.006
Energy difference less than 1.0e-8, stopping DMRG
Ground state energy: -8.320148853431089
Overlap: 1.0
Final Maximum Bond Dimension: 4


In [ ]:
# Creating Time-Dependent Wave Function

omega = 100
periods = 40
steps_per_period = 10

T = 2π/omega
tau = T/steps_per_period
ts = [tau*(n-0.5) for n in 1:steps_per_period]

driven_t0(t) = t0*cos(omega*t)

gate_sets = [build_trotter_gates(sites, tau, driven_t0(t), g, bc) for t in ts]

steps = periods*steps_per_period + 1
times = [tau*i for i in 0:(steps-1)]

psi_t = copy(psi0)

overlaps = zeros(Float64, steps)
maxdims = zeros(Int, steps)

overlaps[1] = abs2(inner(psi_t, psi0))
maxdims[1] = maxlinkdim(psi_t)

for i in 2:steps
    gates = gate_sets[mod1(i-1, steps_per_period)]
    psi_t = apply(gates, psi_t; cutoff=cutoff, maxdim=maximum(maxdim))
    normalize!(psi_t)
    overlaps[i] = abs2(inner(psi_t, psi0))
    maxdims[i] = maxlinkdim(psi_t)
end

Final Overlap: 0.6614750201439072


In [ ]:
# Autocorrelation

operator = "Z"
ind = 1

O = op(operator, sites[ind])

psi_t = copy(psi0)
phi_t = apply(O, psi0)

ac = zeros(ComplexF64, steps)
ac[1] = inner(phi_t, apply(O, psi_t))

for i in 2:steps
    gates = gate_sets[mod1(i-1, steps_per_period)]
    psi_t = apply(gates, psi_t; cutoff=cutoff, maxdim=maximum(maxdim))
    normalize!(psi_t)

    phi_t = apply(gates, phi_t; cutoff=cutoff, maxdim=maximum(maxdim))
    normalize!(phi_t)

    ac[i] = inner(phi_t, apply(O, psi_t))
end

real_ac = real.(ac);
im_ac = imag.(ac);
abs_ac = abs.(ac);

In [31]:
# Print to .csv File

output = DataFrame(
    "time" => times,
    "overlap" => overlaps,
    "maxdim" => maxdims,
    "real_ac"=> real_ac,
    "im_ac" => im_ac
    )

fname = "L$(L)_$(bc)_g$(g)_omega$(omega)_N$(periods).csv"
CSV.write(fname, output)

println("File Created: ", fname)

File Created: L4_OBC_g1_omega100_N40.csv
